In [1]:
# Imports!

from twitterscraper import query_tweets
import tweepy
import datetime
import json, codecs
import jsonpickle
import GetOldTweets3 as got
import math

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

INFO: {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'}


In [2]:
# Load Twitter API credentials
# Credit to Temple Moore for authentication help and setting up tweepy

with open('../credentials.json') as creds:
    info = json.load(creds)
    consumer_key = info['CONSUMER_KEY']
    consumer_secret = info['CONSUMER_SECRET']
    access_token = info['ACCESS_TOKEN']
    access_secret = info['ACCESS_SECRET']

In [3]:
# Authenticate to Twitter

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

try:
    api.verify_credentials()
    print("Autentication Successful")
except:
    print("Error during authentication")

Autentication Successful


In [4]:
user = api.me()
print(user.name)
print(api.rate_limit_status()['resources']['search'])

DC DSI Client Project - Problem 1
{'/search/tweets': {'limit': 180, 'remaining': 180, 'reset': 1564623542}}


In [5]:
# Function written by using code modified from Jasmine Vasandani via Temple Moore
# With the help of got.manager, get all the tweets from specified timeframe, with specified query and search area
# default distance of 20km, default maxtweets of 1500
# Store all the collected tweets in DF

def twit_scrape(since, until, query, near, dist='20km', maxtweets = 3000):
    tweets=[]
    final_df=pd.DataFrame()
    tweetCriteria = got.manager.TweetCriteria().setSince(since).setUntil(until).setQuerySearch(query).setNear(near).setWithin(dist).setMaxTweets(maxtweets).setLang('en')
    print(tweetCriteria)
    tweet = got.manager.TweetManager.getTweets(tweetCriteria)
    for i in range(len(tweet)):
        tweet_dict = {}
        tweet_dict['id'] = tweet[i].id
        tweet_dict['username'] = tweet[i].username
        tweet_dict['date'] = tweet[i].date
        tweet_dict['text'] = tweet[i].text
        tweet_dict['hashtags'] = tweet[i].hashtags
        tweet_dict['geo'] = tweet[i].geo
#         tweet_dict['lat'] = near[0]      # This doesn't work as intended - better not to use
#         tweet_dict['long'] = near[1]     # This doesn't work as intended - better not to use
        tweets.append(tweet_dict)

    df1=pd.DataFrame(tweets)
    final_df = pd.concat([df1.reset_index(), final_df.reset_index()], sort = True)
    print("Shape of DataFrame:", final_df.drop('index', axis = 1).shape)
    return final_df.drop('index', axis = 1)

In [6]:
# Testing the above function to scrape tweets with passed parameters
houston = twit_scrape(since='2017-08-24', until='2017-08-27', query='hurricane, OR harvey, OR flood, OR flooding',
            near='29.7604, -95.3698')

Shape of DataFrame: (406, 8)


In [7]:
houston.shape

(406, 8)

In [8]:
houston.head()

,date,geo,hashtags,id,lat,long,text,username
0,2017-08-26 23:42:09+00:00,,,901590620623523840,2,9,"Power outages, flight cancellations after Harv...",BLifeAldine
1,2017-08-26 23:42:08+00:00,,,901590618845143042,2,9,"Greenspoint, still in flood recovery, waits un...",BLifeAldine
2,2017-08-26 23:41:48+00:00,,,901590533440761856,2,9,Video: Lucky driver escapes flooded car after ...,BLifeAcresHomes
3,2017-08-26 23:41:02+00:00,,,901590339353444354,2,9,The bayou has risen since Hurricane Harvey has...,krivochenco
4,2017-08-26 23:36:38+00:00,,#cicadas #harvey #westbury,901589234938675200,2,9,Almost dusk. Listen audio #cicadas #harvey #we...,DoBianchi


In [9]:
# def twit_scrape_grid(coords):
#     for i in range(len(coords)):
#         print(i)
#         twit_scrape()

In [10]:
# This saves the dataframe to a csv - change the name for each individual scrape
# houston.to_csv('./data/harveyhouston_20km.csv', index = False)